In [1]:
import pytesseract
import numpy as np
import imutils
import cv2
from PIL import Image

In [2]:
class PyImageSearchANPR:
    def __init__(self, ar=3.5):
        self.aspect_ratio_min = ar  # ratio width / height
        self.aspect_ratio_max = 2 * ar
        self.debug = False

    def debug_imshow(self, title, image):
        if self.debug:
            cv2.imshow(title, image)
            cv2.waitKey(0)

    def locate_license_plate_candidates(self, gray):
        # perform morpholgical operation, reveal dark regions on light background
        rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (14, 5))
        blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)
        self.debug_imshow("Blackhat", blackhat)

        # Find regions in imagae that are light
        squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        light = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, squareKern)
        light = cv2.threshold(light, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        self.debug_imshow("Light Regions", light)

        # compute teh scharr gradient representation of the blackhat
        # image in the x-direction and then scale the result back to
        # the range [0, 255]
        gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        gradX = np.absolute(gradX)
        (minVal, maxVal) = (np.min(gradX), np.max(gradX))
        gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
        gradX = gradX.astype("uint8")
        self.debug_imshow("Scharr", gradX)

        # smooth the group regions that may contain boundries to license plate characters
        # blur the gradient representation, applying a closing
        # operation, and threshold the image using Otsu's method
        gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
        gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
        thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        thresh = cv2.erode(thresh, None, iterations=1)
        thresh = cv2.dilate(thresh, None, iterations=1)
        self.debug_imshow("Grad Thresh", thresh)

        # take the bitwise and between the threshold result and the
        # light regions of the image
        thresh = cv2.bitwise_and(thresh, thresh, mask=light)
        thresh = cv2.dilate(thresh, None, iterations=2)
        thresh = cv2.erode(thresh, None, iterations=1)
        self.debug_imshow("final", thresh)

        # find contours in the threshold image
        contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contours = imutils.grab_contours(contours)
        return contours

    def locate_license_plate(self, gray, candidates):
        contours = []
        roi = []
        for c in candidates:
            # compute the bonding box of the contour and the use
            # the bounding box to derive the aspect ratio
            (x, y, w, h) = cv2.boundingRect(c)
            ar = w / float(h)

            if self.aspect_ratio_min <= ar < self.aspect_ratio_max:
                contours.append(c)
                roi.append(cv2.threshold(gray[y:y + h, x:x + w], 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1])
        return roi, contours

    def find_and_ocr(self, image):
        lpText = []
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        candidates = self.locate_license_plate_candidates(gray)

        (plates, contours) = self.locate_license_plate(gray, candidates)
        for i, plate in enumerate(plates):
            self.debug_imshow("License Plate", plate)
            sr = cv2.dnn_superres.DnnSuperResImpl_create()
            path = "EDSR_x2.pb"
            (x, y, w, h) = cv2.boundingRect(contours[i])
            sr.readModel(path)
            sr.setModel("edsr", 2)

            result = sr.upsample(image[y:y + h, x:x + w])
            result = cv2.cvtColor(result, cv2.COLOR_RGBA2GRAY)
            result = cv2.threshold(result, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            result = cv2.medianBlur(result, 5)
            lpText.append(pytesseract.image_to_string(result, config="--psm 13"))
            self.debug_imshow("License Plate", result)
        return lpText, contours

In [6]:
visualise = False
for i in range(1, 31):    
    image = cv2.imread(f'number-plate-benchmark-dataset/images/{i}.png')
    with open(f'number-plate-benchmark-dataset/plates/{i}.txt') as f:
        groundTruth = f.readline()
    anpr = PyImageSearchANPR()
    image = imutils.resize(image, width=1280)
    (found_texts, contours) = anpr.find_and_ocr(image)
    if visualise:
        for i in range(len(found_texts)):
            box = cv2.boxPoints(cv2.minAreaRect(contours[i]))
            box = box.astype("int")
            cv2.drawContours(image, [box], -1, (0, 255, 0), 2)
            (x, y, w, h) = cv2.boundingRect(contours[i])
            cv2.putText(image,
                        "".join([c if c.isalnum() else "" for c in found_texts[i]]).strip(),
                        (x, y - 15),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.75,
                        (0, 255, 0),
                        2)
            print("".join([c if c.isalnum() else "" for c in found_texts[i]]).strip().replace(' ', ''))
        Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).show()
    print(f'Ground truth: {groundTruth}, predicted: {["".join([c if c.isalnum() else "" for c in text]).replace(" ", "") for text in found_texts]}')

Ground truth: WD3677A, predicted: ['wi1AZ7A', 'reeel', 'genee']
Ground truth: WND53600, predicted: ['PORTO']
Ground truth: WD2698H, predicted: ['TWhJAVILA']
Ground truth: RSR85NK, predicted: ['orePg', '']
Ground truth: WMLAE62, predicted: ['CHMALS']
Ground truth: WOT33486, predicted: ['']
Ground truth: WL4871C
, predicted: ['IWPNMRA', 'reeAG7']
Ground truth: WU7229F, predicted: ['i', 'oe']
Ground truth: WU9343A
, predicted: ['RWU9IBA', 'Nefe']
Ground truth: WL8671K, predicted: ['Awionai']
Ground truth: DW4KG70, predicted: ['DW4KG70', 'YY']
Ground truth: SCI12697, predicted: ['SCT12697', 'a']
Ground truth: WR791CK, predicted: ['AWR791CK', 'qe']
Ground truth: DW6MH65, predicted: ['yee']
Ground truth: DW1CY32, predicted: ['OWICY32']
Ground truth: WE081XH, predicted: ['i', 'BMEO81XH']
Ground truth: WD5982P, predicted: ['EE', 'aow', 'egree']
Ground truth: PO1UH08, predicted: ['APO1UHO84', '', 'Ppne']
Ground truth: WZ2154Y, predicted: []
Ground truth: FGW10616, predicted: []
Ground truth: PN

In [3]:
with open('number-plate-benchmark-dataset/plates/1.txt') as f:
    print(f.readline())
    

WD3677A
